個々のメールを特徴量ベクトルに変換するデータ準備パイプラインを書く。準備パイプラインは、メールで使われ得る個々の単語の有無を示す（疎）ベクトルを作る。たとえば、すべてのメールに含まれている単語が"Hello"、"how"、"are"、"you"の4語だけだとすると、"Hello you Hello Hello you"というメールは、[1, 0, 0, 1]というベクトルに変換される（Helloはある、howはない、areはない、youはあるという意味）。[3, 0, 0, 2] を生成して、個々の単語の出現数を数えてもよい。

準備パイプラインには、メールヘッダを取り除くかどうか、個々のメールを小文字に変換するかどうか、記号を取り除くかどうか、すべてのURLを"URL"に変換するかどうか、すべての数値を"NUMBER"に変換するかどうか、ステミング（stemming：つまり、単語の変化を取り除くこと。そのためのPythonライブラリが作られている）をするかどうかを指定するハイパーパラメータを追加する。

最後に複数の分類器を試し、再現率と適合率の両方が高い優れたスパム分類器を構築できるかどうかを考える。

In [7]:
def detect_mime_type(file):
  mime_type = 'text'

  while True:
    line = file.readline()
    # print(line)

    is_content_type = line.startswith("Content-Type:")
    if not is_content_type:
      continue

    is_html = "text/html" in line
    if is_html:
      mime_type = 'html'
    break
    
  return mime_type

In [16]:
def _read(path):
  with open(path, mode="r") as f:
    mime_type = detect_mime_type(f)
    print(mime_type)


In [3]:
files = os.listdir("./samples/easy_ham")
files[:5]

['2170.78c282a5e417d6d231dc75aa8588ebb7',
 '0461.7609029b07acf65f40bd93882aa35887',
 '1077.5967a2892063a3470f32596403b095aa',
 '1000.348e8a82897b05d584de019f4dd5c7e2',
 '2125.ee1e93366c2199584cfbd4cbb353d918']

In [17]:
_read("./samples/easy_ham/" + files[0])

text


In [18]:
_read("./samples/spam/0001.bfc8d64d12b325ff385cca8d07b84288")

html


In [19]:
import re

def _read(path):
  with open(path, mode="r") as file:
    mime_type = detect_mime_type(file)
    print(mime_type)

    # skip row for `from` and headers
    file.readline()
    while True:
      line = file.readline()
      if re.search("\A.*:\z", line):
        # when 
        continue 
      elif line == "":
        break

    if mime_type == "text":
      return file.readline()


In [20]:
_read("./samples/easy_ham/2170.78c282a5e417d6d231dc75aa8588ebb7")

text


''

"X-Spam-Level:" の内容を削除したのか、空の行が入っている。他のパターンがないか確認

In [4]:
!cd "./samples"
!rg --multiline  "^[^\s]*:[\s]*?\n\n" | rg --invert-match "(X-Spam-Level:)" | rg --invert-match ".+/[\d]+\..+:$"

samples/easy_ham/1399.5ee3b9902af076cc63e0c4e612f99ba0:BlogStart: 
samples/easy_ham/2457.5fba5a23aa428e3201526f7c978c88c6:medium: 
samples/easy_ham/2326.73396f0421612c54d1c9727ed953582e:Potty.: 
samples/easy_ham/2198.161bc9ce9471f604149d2468b2bc198d:M3: 
samples/spam/0062.41f708df91642411f0fd8f91e28d3521: 
samples/hard_ham/0155.4b7d1d147b6a15245d7ce572ef804b74:BlogStart: 
samples/hard_ham/0138.6f5a6164234f79dfaaa530134395bef1:Software: 
samples/hard_ham/0154.b7e55edcd1795f418e9949a9092b8018:Links: 
samples/hard_ham/0154.b7e55edcd1795f418e9949a9092b8018:Links: 
samples/hard_ham/0154.b7e55edcd1795f418e9949a9092b8018:Links: 
samples/hard_ham/0154.b7e55edcd1795f418e9949a9092b8018:Links: 
samples/hard_ham/0154.b7e55edcd1795f418e9949a9092b8018:Links: 
samples/hard_ham/0182.b7ddf319da0b2846f1f78a20e7a6fc9c:::::::::::::::::::::::::::::::	
samples/hard_ham/0182.b7ddf319da0b2846f1f78a20e7a6fc9c: 
samples/spam/0326.80f15e07265a22b78068bab5b56b01c7:Country:											
